In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from muxcnn.utils import load_params, load_img
#from muxcnn.hecnn_par import *
#from muxcnn.comparator import ApprRelu
import torch
from muxcnn.resnet_muxconv import ResNetFHE

## Torch model

In [3]:
from muxcnn.models.ResNet20 import ResNet, BasicBlock

model = ResNet(BasicBlock,[1,1,1])
model.eval() ########## 필수! 
load_params(model, fn_param="./ResNet8.pt",device='cpu')

In [4]:
img_tensor = load_img("./cute.jpg", hi=32, wi=32)

In [5]:
print(model(img_tensor))

before softmax tensor([[ -9.3005, -11.0586,  -2.9741,  -2.9179,  -9.3500,   4.1474,  -6.8721,
          -3.6999,  -8.9446,  -7.6901]], grad_fn=<AddmmBackward0>)
tensor([[-1.3450e+01, -1.5208e+01, -7.1236e+00, -7.0674e+00, -1.3500e+01,
         -2.0791e-03, -1.1022e+01, -7.8494e+00, -1.3094e+01, -1.1840e+01]],
       grad_fn=<LogSoftmaxBackward0>)


# FHE

In [14]:
from time import time

In [18]:
fhe_model = ResNetFHE(model, alpha=12)

functions set
degrees = [15, 15, 15, 15], margin = 0.01, eps = 0.02


In [19]:
t0 = time()
result = fhe_model.forward_early(img_tensor)
print(f"{time() - t0:.4f} s")

[MultParConv] (hi,wi,ci,ki,ti,pi) =(32,32, 3, 1, 3,  8)
[MultParConv] (ho,wo,co,ko,to,po) =(32,32,16, 1,16,  2)
[MultParConv] q = 2
1.9505 s
1.9635 s


In [20]:
t0 = time()
result = fhe_model(img_tensor)
print(f"{time() - t0:.4f} s")

[MultParConv] (hi,wi,ci,ki,ti,pi) =(32,32, 3, 1, 3,  8)
[MultParConv] (ho,wo,co,ko,to,po) =(32,32,16, 1,16,  2)
[MultParConv] q = 2
1.9421 s
[MultParConv] (hi,wi,ci,ki,ti,pi) =(32,32,16, 1,16,  2)
[MultParConv] (ho,wo,co,ko,to,po) =(32,32,16, 1,16,  2)
[MultParConv] q = 8
7.5915 s
[MultParConv] (hi,wi,ci,ki,ti,pi) =(32,32,16, 1,16,  2)
[MultParConv] (ho,wo,co,ko,to,po) =(32,32,16, 1,16,  2)
[MultParConv] q = 8
7.5881 s
[MultParConv] (hi,wi,ci,ki,ti,pi) =(32,32,16, 1,16,  2)
[MultParConv] (ho,wo,co,ko,to,po) =(16,16,32, 2, 8,  4)
[MultParConv] q = 16
14.5817 s
[MultParConv] (hi,wi,ci,ki,ti,pi) =(16,16,32, 2, 8,  4)
[MultParConv] (ho,wo,co,ko,to,po) =(16,16,32, 2, 8,  4)
[MultParConv] q = 8
6.9168 s
[MultParConv] (hi,wi,ci,ki,ti,pi) =(32,32,16, 1,16,  2)
[MultParConv] (ho,wo,co,ko,to,po) =(16,16,32, 2, 8,  4)
[MultParConv] q = 16
2.1417 s
[MultParConv] (hi,wi,ci,ki,ti,pi) =(16,16,32, 2, 8,  4)
[MultParConv] (ho,wo,co,ko,to,po) =( 8, 8,64, 4, 4,  8)
[MultParConv] q = 16
13.3325 s
[MultPar

In [21]:
torch.tensor(result[::64][:10])

tensor([ -9.2678, -10.8150,  -3.1036,  -2.9026,  -9.4616,   4.3280,  -6.9752,
         -3.5619,  -8.8842,  -7.5630], dtype=torch.float64)

In [10]:
torch.tensor(result[::64][:10])

tensor([ -9.2678, -10.8150,  -3.1036,  -2.9026,  -9.4616,   4.3280,  -6.9752,
         -3.5619,  -8.8842,  -7.5630], dtype=torch.float64)